In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import category_encoders as ce
import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

lightGBMではなくて、ランダムフォレストを使ってみる。

In [86]:
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')
floor_scores = pd.read_csv('floor_scores.csv')
Floor_scores = pd.read_csv('capital_floor_scores.csv')
addresses = pd.read_csv('addresses.csv')
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')

rent = pd.read_csv('rent.csv')

In [87]:
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')

In [88]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)


In [89]:
X_train

,築年数,面積,契約期間,所在階,全体の階数,最寄り駅,所要時間,所在地,部屋数,L,D,K,S
1382,589.0,28.35,3.0,2.0,12.0,99,4.0,514,1,1,1,1,0
15190,29.0,25.45,2.0,3.0,3.0,417,8.0,659,1,0,0,1,0
28196,365.0,44.55,2.0,7.0,11.0,86,13.0,256,2,0,1,1,0
18762,4.0,33.91,2.0,7.0,13.0,16,4.0,16,2,0,0,1,0
13923,143.0,22.62,2.0,2.0,10.0,385,2.0,370,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,181.0,50.01,2.0,1.0,2.0,61,NaN,304,2,0,1,1,0
19648,0.0,25.80,2.0,2.0,4.0,141,6.0,14,1,0,0,1,0
9845,388.0,36.72,2.0,NaN,NaN,51,NaN,558,1,0,1,1,0
10799,374.0,19.00,0.0,1.0,2.0,154,13.0,133,1,0,0,0,0


ランダムフォレストは欠損値を許してくれないので整形をする。これくらいの数の欠損値であれば削除してもよさそう。

In [90]:
X_train.isna().sum()

築年数        3
面積         0
契約期間      74
所在階      481
全体の階数    481
最寄り駅       0
所要時間     177
所在地        0
部屋数        0
L          0
D          0
K          0
S          0
dtype: int64

In [91]:
#訓練データについて
Train_Data = pd.concat([X_train, y_train],axis=1)
Train_Data = Train_Data.dropna()
X_train = Train_Data[['築年数','面積','契約期間','所在階','全体の階数','最寄り駅','所要時間','所在地','部屋数','L','D','K','S']]
y_train = Train_Data[['賃料']]

#検証データについて
Valid_Data = pd.concat([X_valid, y_valid],axis=1)
Valid_Data = Valid_Data.dropna()
X_valid = Valid_Data[['築年数','面積','契約期間','所在階','全体の階数','最寄り駅','所要時間','所在地','部屋数','L','D','K','S']]
y_valid = Valid_Data[['賃料']]

In [92]:
clf = RandomForestRegressor(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_valid)
print(r2_score(y_valid,y_pred))

/var/folders/qh/20ftf7rn5_58_rk3l25kt1nr0000gn/T/ipykernel_5212/2176165941.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


0.8594671351795132


このランダムフォレストをハイパーパラメータチューニングする

In [93]:
#訓練データについて
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent
Train_Data = pd.concat([X_train, y_train],axis=1)
Train_Data = Train_Data.dropna()
X_train = Train_Data[['築年数','面積','契約期間','所在階','全体の階数','最寄り駅','所要時間','所在地','部屋数','L','D','K','S']]
y_train = Train_Data[['賃料']]

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

#検証データについて
X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)
Valid_Data = pd.concat([X_valid, y_valid],axis=1)
Valid_Data = Valid_Data.dropna()
X_valid = Valid_Data[['築年数','面積','契約期間','所在階','全体の階数','最寄り駅','所要時間','所在地','部屋数','L','D','K','S']]
y_valid = Valid_Data[['賃料']]

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 5, 300),
        #'max_features'      : [i for i in range(1,train_data_bs.shape[1])],
        'random_state': 0,
        'n_jobs': 1,
        'min_samples_split': trial.suggest_int('min_samples_split', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 100),
    }

    clf = RandomForestRegressor(
                                **params,#ここでアスタリスク２つをつけないとエラーになる。
                                )

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    loss = mean_squared_error(y_valid, y_pred, squared=False)
    return loss

In [94]:
optuna.logging.disable_default_handler()
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=50)
study.best_params

/var/folders/qh/20ftf7rn5_58_rk3l25kt1nr0000gn/T/ipykernel_5212/4243251184.py:33: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
Trial 0 failed because of the following error: ValueError('Input X contains NaN.\nRandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values')
Traceback (most recent call l

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values